In [ ]:
# Check the GPU status
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
# Import the necessary Library and Files
from osgeo import gdal_array
from osgeo import gdal
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt
from sklearn.metrics import r2_score
from pandas import DataFrame
from pandas import concat
import pandas as pd


In [ ]:
# Load the test data set¶
filename="/content/drive/My Drive/GIS/test_dataset_canada.dat"
raster = gdal.Open(filename)
rasterArray = raster.ReadAsArray()
tb1 = rasterArray[0,0:rasterArray.shape[1],0:rasterArray.shape[2]]
tb2 = rasterArray[1,0:rasterArray.shape[1],0:rasterArray.shape[2]]
tb3 = rasterArray[2,0:rasterArray.shape[1],0:rasterArray.shape[2]]
tb4 = rasterArray[3,0:rasterArray.shape[1],0:rasterArray.shape[2]]
tb5 = rasterArray[4,0:rasterArray.shape[1],0:rasterArray.shape[2]]
tb6 = rasterArray[5,0:rasterArray.shape[1],0:rasterArray.shape[2]]
tb1_reshape=np.reshape(tb1,(tb1.shape[0]*tb1.shape[1],))
tb3_reshape=np.reshape(tb3,(tb3.shape[0]*tb3.shape[1],))
tb4_reshape=np.reshape(tb4,(tb4.shape[0]*tb4.shape[1],))
tb5_reshape=np.reshape(tb5,(tb5.shape[0]*tb5.shape[1],))
tb6_reshape=np.reshape(tb6,(tb6.shape[0]*tb6.shape[1]))
tb2_reshape=np.reshape(tb2,(tb2.shape[0]*tb2.shape[1],))

In [ ]:
test_data=np.stack((tb1_reshape,tb3_reshape,tb4_reshape, tb5_reshape, tb6_reshape),axis=-1)
target=tb2_reshape
n_lag=1
n_feature=5

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [ ]:
test_reframed=series_to_supervised(test_data,n_lag,1)
test_values=test_reframed.values

In [ ]:
test_y=target[n_lag:]
test_y.shape

In [ ]:
test_X=test_reframed.values
test_X = np.reshape(test_X, (test_X.shape[0], n_lag+1, n_feature))

In [ ]:
print(test_X.shape)
print(test_y.shape)

In [ ]:
# load pretrained lstm models form disk
from keras.models import model_from_json
json_file = open('../trained_models/LSTM-model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("../trained_models/LSTM-model.h5")
print("Loaded model from disk")

In [ ]:
loaded_model.compile(loss='mse', optimizer='adam')

In [ ]:
#whole test data performance evaluations
pred_y=loaded_model.predict(test_X)
print("r2 score: %.6f" %r2_score(test_y,pred_y))
rmse = sqrt(mean_squared_error(test_y,pred_y))
print('Test RMSE: %.3f' % rmse)
# calculate MAE
mae = mean_absolute_error(test_y,pred_y)
print('Test MAE: %.3f' % mae)'''

In [ ]:
j=0
case1_x=[]
case1_y=[]
case2_x=[]
case2_y=[]
case3_x=[]
case3_y=[]
case4_x=[]
case4_y=[]
case5_x=[]
case5_y=[]
case6_x=[]
case6_y=[]
case7_x=[]
case7_y=[]
case8_x=[]
case8_y=[]
case9_x=[]
case9_y=[]
for i in range(tb1.size):
    if(tb1_reshape[i]<tb2_reshape[i]<tb3_reshape[i]):
        case1_x.append([tb1_reshape[i],tb3_reshape[i],tb4_reshape[i],tb5_reshape[i],tb6_reshape[i]])
        case1_y.append(tb2_reshape[i])
    elif(tb1_reshape[i]<tb2_reshape[i]==tb3_reshape[i]):
        case2_x.append([tb1_reshape[i],tb3_reshape[i],tb4_reshape[i],tb5_reshape[i],tb6_reshape[i]])
        case2_y.append(tb2_reshape[i])
    elif(tb1_reshape[i]<tb2_reshape[i]>tb3_reshape[i]):
        case3_x.append([tb1_reshape[i],tb3_reshape[i],tb4_reshape[i],tb5_reshape[i],tb6_reshape[i]])
        case3_y.append(tb2_reshape[i])
    elif(tb1_reshape[i]==tb2_reshape[i]<tb3_reshape[i]):
        case4_x.append([tb1_reshape[i],tb3_reshape[i],tb4_reshape[i],tb5_reshape[i],tb6_reshape[i]])
        case4_y.append(tb2_reshape[i])
    elif(tb1_reshape[i]==tb2_reshape[i]==tb3_reshape[i]):
        case5_x.append([tb1_reshape[i],tb3_reshape[i],tb4_reshape[i],tb5_reshape[i],tb6_reshape[i]])
        case5_y.append(tb2_reshape[i])
    elif(tb1_reshape[i]==tb2_reshape[i]>tb3_reshape[i]):
        case6_x.append([tb1_reshape[i],tb3_reshape[i],tb4_reshape[i],tb5_reshape[i],tb6_reshape[i]])
        case6_y.append(tb2_reshape[i])
    elif(tb1_reshape[i]>tb2_reshape[i]<tb3_reshape[i]):
        case7_x.append([tb1_reshape[i],tb3_reshape[i],tb4_reshape[i],tb5_reshape[i],tb6_reshape[i]])
        case7_y.append(tb2_reshape[i])
    elif(tb1_reshape[i]>tb2_reshape[i]==tb3_reshape[i]):
        case8_x.append([tb1_reshape[i],tb3_reshape[i],tb4_reshape[i],tb5_reshape[i],tb6_reshape[i]])
        case8_y.append(tb2_reshape[i])
    elif(tb1_reshape[i]>tb2_reshape[i]>tb3_reshape[i]):
        case9_x.append([tb1_reshape[i],tb3_reshape[i],tb4_reshape[i],tb5_reshape[i],tb6_reshape[i]])
        case9_y.append(tb2_reshape[i])

In [ ]:
def prepare_case(case1_x,case1_y):
    a=np.array(case1_x)
    b=np.array(case1_y)
    assert a.shape[0]==b.shape[0]
    case1_x_data=series_to_supervised(a,n_lag,1)
    y=b[n_lag:]
    case1_x_data=case1_x_data.values
    x = np.reshape(case1_x_data, (case1_x_data.shape[0], n_lag+1, n_feature))
    return (x,y)

In [ ]:
def predict_case(x,y):
    pred_y=loaded_model.predict(x)
    print("r2 score %",r2_score(y,pred_y))
    rmse = sqrt(mean_squared_error(y,pred_y))
    print('Test RMSE: %.3f' % rmse)
    # calculate MAE
    mae = mean_absolute_error(y,pred_y)
    print('Test MAE: %.3f' % mae)
    return(pred_y)

In [ ]:
x,y=prepare_case(case1_x,case1_y)
y1_hat=predict_case(x,y)
x3,y3=prepare_case(case3_x,case3_y)
y3_hat=predict_case(x3,y3)
x7,y7=prepare_case(case7_x,case7_y)
y7_hat=predict_case(x7,y7)
x9,y9=prepare_case(case9_x,case9_y)
y9_hat=predict_case(x9,y9)